# Image generation

In Part 1 of this course, we focused mainly on models that were useful for classification. However, many applications require generating much higher dimensional results, such as images and sentences. Examples include:
* Text: neural translation, text to speech, image captioning
* Image: Segmentation, artistic filters, image sharpening and cleaning

In [1]:
from utils2 import *

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%matplotlib inline
from time import time
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
from keras import metrics

from vgg16_avg import VGG16_Avg


    
class Evaluator(object):
    def __init__(self, function, shape):
        self.function = function
        self.shape = shape
    
    def loss(self, x):
        loss_, self.grad_values = self.function([x.reshape(self.shape)])
        return loss_.astype(np.float64)
    
    def grads(self, x):
        return self.grad_values.flatten().astype(np.float64)
    
def plots(ims, figsize=(12,6), rows=1, cols=None, interp=None, titles=None, cmap=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        if titles is not None:
            sp.set_title(titles[i], fontsize=18)
        plt.imshow(ims[i], interpolation=interp, cmap=cmap)
        plt.axis('off') # New
        plt.subplots_adjust(hspace = 0.500) # New
    return

def solve_image(eval_obj, niter, x):
    for i in range(niter):
        x, min_val, info = fmin_l_bfgs_b(eval_obj.loss, x.flatten(),
                                         fprime=eval_obj.grads, maxfun=20)
        x = np.clip(x, -127,127)
        print('Current loss value:', min_val)
        imsave(f'{result_path}/res_at_iteration_{i}.png', deproc(x.copy(), shp)[0])
    return x

imagenet_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
rn_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
rand_img = lambda shape: np.random.uniform(-2.5, 2.5, shape) / 100
preproc = lambda x: (x - rn_mean)[:, :, :, ::-1]
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + imagenet_mean, 0, 255)


In [3]:
def gram_matrix(x):
    # We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # The dot product of this with its transpose shows the correlation 
    # between each pair of channels
    numerator = K.dot(features, K.transpose(features)) 
    denominator = x.get_shape().num_elements()
    output = numerator / denominator
    return output 
def get_style_loss(x, targ): return metrics.mse(gram_matrix(x), gram_matrix(targ))    


In [3]:
limit_mem()

### Setup the path

In [4]:
result_path = '/home/ubuntu/data/results/midnightinparis'
content_path = '/home/ubuntu/data/lesson1/newtrain/midnight.jpg'
style_path = '/home/ubuntu/data/starry_night.jpg'

### define functions and libraries

In [5]:
def get_content_arr(fpath):
    img = Image.open(fpath)
    #plt.imshow(img)
    content_arr = preproc(np.expand_dims(img, 0))
    shape = content_arr.shape
    #print (shape)
    return content_arr, shape

In [6]:
def get_starting_point(shape):
    x = rand_img(shape)
    #plt.imshow(x[0])
    return x

In [7]:
def get_content(content_path):
    content_arr, shape = get_content_arr(content_path)
    starting_pt = get_starting_point(shape)
    return content_arr, starting_pt

In [8]:
def get_style_arr(style_path):
    img = Image.open(style_path)
    img = img.resize(np.divide(img.size, 2).astype('int32'))
    style_arr = preproc(np.expand_dims(img, 0))
    #plt.imshow(img)
    return style_arr
    

In [9]:
def fit_style(content_path, style_arr):
    img =Image.open(content_path)
    w, h = img.size
    style_arr = style_arr[:, :h, :w]
    
    style_cropped = deproc(style_arr, style_arr.shape)[0].astype('uint8')
    return style_cropped

In [10]:
def get_style(style_path, content_path):
    style_arr = get_style_arr(style_path)
    style_cropped = fit_style(content_path, style_arr)
    return style_arr, style_cropped

In [11]:
def get_content_loss(vgg16avg, content_arr):
    content_layer = vgg16avg.get_layer('block4_conv2').output
    content_model = Model(vgg16avg.input, content_layer)
    content_target = K.variable(content_model.predict(content_arr))
    content_loss = metrics.mse(content_layer, content_target)
    return content_loss

In [12]:
def get_style_loss(vgg16avg, style_arr):
    outputs = {l.name: l.output for l in vgg16avg.layers}
    style_layers = [outputs['block{}_conv1'.format(i)] for i in range(1, 6)]
    style_model = Model(vgg16avg.input, style_layers)
    style_targets = [K.variable(o) for o in style_model.predict(style_arr)]
    layer_losses = [get_style_loss(la[0], tla[0]) for la, tla in zip(style_layers, style_targets)]
    style_loss = sum(layer_losses)
    return style_loss

In [13]:
def merge(vgg16avg, content_loss, style_loss, shape):
    loss = content_loss/ 10. + style_loss
    grads = K.gradients(loss, vgg16avg.input)
    transfer_fn = K.function([vgg16avg.input], [loss]+grads) 
    evaluator = Evaluator(transfer_fn, shape)
    return evaluator

In [14]:
def neural_style_model(content_arr, style_arr):
    vgg16avg = VGG16_Avg(include_top = False)    
    content_loss = get_content_loss(vgg16avg, content_arr)
    shp = style_arr.shape
    style_vgg16avg = VGG16_Avg(include_top=False, input_shape=shp[1:])
    style_loss = get_style_loss(style_vgg16avg, style_arr)
    shape = content_arr.shape
    evaluator = merge(vgg16avg, content_loss, style_loss, shape)
    return evaluator

In [15]:
def fit(evaluator, nb_iterations, starting_pt, shape):
    start = int(time())
    x_final = solve_image(evaluator, nb_iterations, starting_pt)

### create a model

In [17]:
content_arr, starting_pt = get_content(content_path)
style_arr, style_cropped = get_style(style_path, content_path)
#evaluator = neural_style_model(content_arr, style_arr)
#fit(evaluator, 10, starting_pt, content_arr.shape)
#generate ()

In [ ]:
## merge
loss = content_loss/ 10. + style_loss
grads = K.gradients(loss, vgg16avg.input)
transfer_fn = K.function([vgg16avg.input], [loss]+grads) 
evaluator = Evaluator(transfer_fn, shape)

In [ ]:
x = rand_img(shape)

In [ ]:
x = solve_image(evaluator, 10, x)

In [ ]:

# plot images
images = [Image.open(content_path), Image.open(style_path), starting_pt[0]]
plots(images, rows=2, cols=2, titles=["content", "style", "initial"], figsize=(12,8))

create a random image

Our first step is to grab an image

We need to subtract the mean of each channel of the imagenet data and reverse the order of RGB->BGR since those are the preprocessing steps that the VGG authors did - so their model won't work unless we do the same thing.

We can do this in one step using *broadcasting*, which is a topic we'll be returning to many times during this course.

In [ ]:
img_arr = preproc(np.expand_dims(np.array(img), 0))
shp = img_arr.shape

When we generate images from this network, we'll need to undo the above preprocessing in order to view them.

In [ ]:
deproc = lambda x,s: np.clip(x.reshape(s)[:, :, :, ::-1] + rn_mean, 0, 255)

#### Broadcasting examples

In [ ]:
np.array([1,2,3]) - 2

In [ ]:
np.array([2,3]).reshape(1,1,1,2)

In [ ]:
np.array([2,3]).reshape(1,1,2,1)

In [ ]:
a = np.random.randn(5,1,3,2)
b = np.random.randn(2)
(a-b).shape

### Recreate input

The first step in style transfer is understanding how to recreate an image from noise based on "content loss", which is the amount of difference between activations in some layer. In earlier layes, content loss is very similar to per-pixel loss, but in later layers it is capturing the "meaning" of a part of an image, rather than the specific details.

To do this, we first take a CNN and pass an image through it. We then pass a "noise image" (i.e. random pixel values) through the same CNN. At some layer, we compare the outputs from it for both images. We then use a MSE to compare the activations of these two outputs. 

The interesting part is that now, instead of updating the parameters of the CNN, we update the pixels of the noisy image. In other words, our goal is to alter the noisy image so as to minimize the difference between the original image's output at some convolutional layer with the output of the noisy image at the same layer.

In order to construct this architecture, we're going to be working with keras.backend, which is an abstraction layer that allows us to target both theano and tensorflow with the same code.

The CNN we'll use is VGG16, but with a twist. Previously we've always used Vgg with max pooling, and this was useful for image classification. It's not as useful in this case however, because max pooling loses information about the original input area. Instead we will use average pooling, as this does not throw away as much information.

In [ ]:
model = VGG16_Avg(include_top=False)

In [ ]:
??VGG16_Avg

Here we're grabbing the activations from near the end of the convolutional model).

In [ ]:
layer = model.get_layer('block5_conv1').output

And let's calculate the target activations for this layer:

In [ ]:
layer_model = Model(model.input, layer)
targ = K.variable(layer_model.predict(img_arr))

In our implementation, we need to define an object that will allow us to separately access the loss function and gradients of a function, since that is what scikit-learn's optimizers require.

We'll define our loss function to calculate the mean squared error between the two outputs at the specified convolutional layer.

In [ ]:
loss = metrics.mse(layer, targ)
grads = K.gradients(loss, model.input)
fn = K.function([model.input], [loss]+grads)
evaluator = Evaluator(fn, shp)

Now we're going to optimize this loss function with a deterministic approach to optimization that uses a line search, which we can implement with sklearn's `fmin_l_bfgs_b` funtionc. 

Next we need to generate a random image.

In [ ]:
rand_img = lambda shape: np.random.uniform(-2.5, 2.5, shape)/100
x = rand_img(shp)
plt.imshow(x[0]);

Now we'll run through this optimization approach ten times and train the noise image's pixels as desired.

In [ ]:
iterations=10

In [ ]:
x = solve_image(evaluator, iterations, x)

Our result by comparing output at conv 1 of last block (5) is fairly amorphous, but still easily recognizable as a bird. Notice that the things it has reconstructed particularly well are those things that we expect Vgg16 to be good at recognizing, such as an eye or a beak.

In [ ]:
Image.open(path + '/res_at_iteration_8.png')

If instead we optimized by calculating loss from the output of conv 1 of 4th block, our trained image looks much more like the original. This makes sense because with less transformations to go through, comparing at an earlier layer means that we have a smaller receptive field and the features are more based on geometric details rather than broad features.

In [ ]:
Image.open(path + '/results/res_at_iteration_9.png')

In [ ]:
from IPython.display import HTML
from matplotlib import animation, rc

In [ ]:
fig, ax = plt.subplots()
def animate(i): ax.imshow(Image.open(f'{path}/res_at_iteration_{i}.png'))

The optimizer first focuses on the important details of the bird, before trying to match the background.

In [ ]:
anim = animation.FuncAnimation(fig, animate, frames=10, interval=200)
HTML(anim.to_html5_video())

### Recreate style

Now that we've learned how to recreate an input image, we'll move onto attempting to recreate *style*. By "style", we mean the color palette and texture of an image. Unlike recreating based on content, with style we are not concerned about the actual structure of what we're creating, all we care about is that it captures this concept of "style".

Here are some examples of images we can extract style from.

In [ ]:
def plot_arr(arr): plt.imshow(deproc(arr,arr.shape)[0].astype('uint8'))

In [ ]:
%pwd

In [ ]:
%cd ..


In [ ]:
style = Image.open('data/tigerfacepaint.jpg')
#style = style.resize(np.divide(style.size,3.5).astype('int32')); style
style = style.resize(np.divide(style.size,1.8).astype('int32')); style

In [ ]:
style = Image.open('data/dogfacepaint.jpg')
#style = style.resize(np.divide(style.size,3.5).astype('int32')); style
style = style.resize(np.divide(style.size,1.0).astype('int32')); style

In [ ]:
style = Image.open('data/vangogh.jpg')
#style = style.resize(np.divide(style.size,3.5).astype('int32')); style
style = style.resize(np.divide(style.size,1.92).astype('int32')); style

In [ ]:
style = Image.open('data/starry_night.jpg')
style = Image.open('https://upload.wikimedia.org/wikipedia/commons/c/cd/VanGogh-starry_night.jpg')
#style = style.resize(np.divide(style.size,3.5).astype('int32')); style
style = style.resize(np.divide(style.size,2.1).astype('int32')); style

In [ ]:
style = Image.open('data/sonofman.jpg')
style = style.resize(np.divide(style.size,1.0).astype('int32')); style
#style = style.resize(np.multiply(style.size,1.14).astype('int32')); style

In [ ]:
style = Image.open('data/simpsons.jpg')
style = style.resize(np.divide(style.size,1.04).astype('int32')); style

We're going to repeat the same approach as before, but with some differences.

In [ ]:
def plots(ims, figsize=(12,6), rows=1, cols=None, interp=None, titles=None, cmap=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        if titles is not None:
            sp.set_title(titles[i], fontsize=18)
        plt.imshow(ims[i], interpolation=interp, cmap=cmap)
        plt.axis('off') # New
        plt.subplots_adjust(hspace = 0.500) # New
    return

In [ ]:
style_arr = preproc(np.expand_dims(style,0)[:,:,:,:3])
w, h = img.size
style_arr = style_arr[:, :h, :w]

style_cropped = deproc(style_arr, style_arr.shape)[0].astype('uint8')
plots([img, style_cropped], rows=1, cols=2, titles=["style", "style cropped"], figsize=(12,8))
shp = style_arr.shape
shp

In [ ]:
model = VGG16_Avg(include_top=False, input_shape=shp[1:])
outputs = {l.name: l.output for l in model.layers}

One thing to notice is that we're actually going to be calculating the loss function multiple layers, rather than just one. (Note however that there's no reason you couldn't try using multiple layers in your content loss function, if you wanted to try that).

In [ ]:
layers = [outputs['block{}_conv1'.format(o)] for o in range(1,4)]

In [ ]:
layers_model = Model(model.input, layers)
targs = [K.variable(o) for o in layers_model.predict(style_arr)]

The key difference is our choice of loss function. Whereas before we were calculating mse of the raw convolutional outputs, here we transform them into the "gramian matrix" of their channels (that is, the product of a matrix and its transpose) before taking their mse. It's unclear why this helps us achieve our goal, but it works. One thought is that the gramian  shows how our features at that convolutional layer correlate, and completely removes all location information. So matching the gram matrix of channels can only match some type of texture information, not location information.

In [ ]:
def gram_matrix(x):
    # We want each row to be a channel, and the columns to be flattened x,y locations
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    # The dot product of this with its transpose shows the correlation 
    # between each pair of channels
    return K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

In [ ]:
def style_loss(x, targ): return metrics.mse(gram_matrix(x), gram_matrix(targ))

In [ ]:
loss = sum(style_loss(l1[0], l2[0]) for l1,l2 in zip(layers, targs))
grads = K.gradients(loss, model.input)
style_fn = K.function([model.input], [loss]+grads)
evaluator = Evaluator(style_fn, shp)

We then solve as we did before.

In [ ]:
iterations=10
x = rand_img(shp)

In [ ]:
x = solve_image(evaluator, iterations, x)

Our results are stunning. By transforming the convolutional outputs to the gramian, we are somehow able to update the noise pixels to produce an image that captures the raw style of the original image, with absolutely no structure or meaning.

In [ ]:
Image.open(path + '/res_at_iteration_9.png')

### Style transfer

We now know how to reconstruct an image, as well as how to construct an image that captures the style of an original image. The obvious idea may be to just combine these two approaches by weighting and adding the two loss functions.

In [ ]:
img_arr.shape

In [ ]:
img_h=img_arr.shape[1]

In [ ]:
img_w =img_arr.shape[2]

In [ ]:
w,h = style.size; print (w, h)

In [ ]:
if img_w<w:
    print ("decrease style width from {} to {}".format(w, img_w))
else:
    print ("width is good to go: img_w{}, w{}".format(img_w, w))

In [ ]:
if img_h<h:
    print ("increase img_h from {} to {}".format(img_h, h))
else:
    print ("height is good to go")    

In [ ]:
imghslice = (img_h-h)/2

In [ ]:
imgslice = (img_w-w)/2

In [ ]:
src = img_arr[:,imghslice:h+imghslice,imgslice:w+imgslice]
plot_arr(src)

Like before, we're going to grab a sequence of layer outputs to compute the style loss. However, we still only need one layer output to compute the content loss. How do we know which layer to grab? As we discussed earlier, the lower the layer, the more exact the content reconstruction will be. In merging content reconstruction with style, we might expect that a looser reconstruction of the content will allow more room for the style to have an effect (re: inspiration). Furthermore, a later layer ensures that the image "looks like" the same subject, even if it doesn't have the same details.

In [ ]:
style_layers = [outputs['block{}_conv1'.format(o)] for o in range(1,6)]
content_name = 'block4_conv2'
content_layer = outputs[content_name]

In [ ]:
style_model = Model(model.input, style_layers)
style_targs = [K.variable(o) for o in style_model.predict(style_arr)]

In [ ]:
content_model = Model(model.input, content_layer)
content_targ = K.variable(content_model.predict(src))

Now to actually merge the two approaches is as simple as merging their respective loss functions. Note that as opposed to our previous to functions, this function is producing three separate types of outputs: one for the original image, one for the image whose style we're emulating, and one for the random image whose pixel's we are training. 

One way for us to tune how the reconstructions mix is by changing the factor on the content loss, which we have here as 1/10. If we increase that denominator, the style will have a larger effect on the image, and if it's too large the original content of the image will be obscured by unstructured style. Likewise, if it is too small than the image will not have enough style.

In [ ]:
loss = sum(style_loss(l1[0], l2[0]) for l1,l2 in zip(style_layers, style_targs))
loss += metrics.mse(content_layer, content_targ)/10.
grads = K.gradients(loss, model.input)
transfer_fn = K.function([model.input], [loss]+grads)

In [ ]:
evaluator = Evaluator(transfer_fn, shp)

In [ ]:
iterations=10
x = rand_img(shp)

In [ ]:
path = '/home/ubuntu/data/results/bird'

In [ ]:
x = solve_image(evaluator, iterations, x)

These results are remarkable. Each does a fantastic job at recreating the original image in the style of the artist.

In [ ]:
Image.open(path + '/res_at_iteration_9.png')

In [ ]:
path = '/home/ubuntu/data/results/tigerfacepaint'

In [ ]:
Image.open(path + '/res_at_iteration_19.png')

In [ ]:
path = '/home/ubuntu/data/results/headshot/'

In [ ]:
Image.open(path + 'res_at_iteration_9.png')

In [ ]:
Image.open(path + 'res_at_iteration_9.png')

In [ ]:
Image.open(path + 'res_at_iteration_9.png')

There are lots of interesting additional things you could try, such as the ideas shown here: https://github.com/titu1994/Neural-Style-Transfer .

## Use content loss to create a super-resolution network

So far we've demonstrated how to achieve succesful results in style transfer. However, there is an obvious drawback to our implementation, namely that we're training an image, not a network, and therefore every new image requires us to retrain. It's not a feasible method for any sort of real-time application. Fortunately we can adress this issue by using a fully convolutional network (FCN), and in particular we'll look at this implementation for Super resolution. We are following the approach in [this paper](https://arxiv.org/abs/1603.08155).

In [ ]:
arr_lr = bcolz.open(dpath+'trn_resized_72.bc')[:]
arr_hr = bcolz.open(dpath+'trn_resized_288.bc')[:]

To start we'll define some of the building blocks of our network. In particular recall the residual block (as used in [Resnet](https://arxiv.org/abs/1512.03385)), which is just a sequence of 2 convolutional layers that is added to the initial block input. We also have a de-convolutional layer (also known as a "transposed convolution" or "fractionally strided convolution"), whose purpose is to learn to "undo" the convolutional function. It does this by padding the smaller image in such a way to apply filters on it to produce a larger image.

In [ ]:
def conv_block(x, filters, size, stride=(2,2), mode='same'):
    x = Convolution2D(filters, size, size, subsample=stride, border_mode=mode)(x)
    x = BatchNormalization(axis=1, mode=2)(x)
    return Activation('relu')(x)

In [ ]:
def res_block(ip, nf=64):
    x = conv_block(ip, nf, 3, (1,1))
    x = Convolution2D(nf, 3, 3, border_mode='same')(x)
    x = BatchNormalization(axis=1, mode=2)(x)
    return merge([x, ip], mode='sum')

In [ ]:
def deconv_block(x, filters, size, shape, stride=(2,2)):
    x = Deconvolution2D(filters, size, size, subsample=stride, border_mode='same',
                        output_shape=(None,)+shape)(x)
    x = BatchNormalization(axis=1, mode=2)(x)
    return Activation('relu')(x)

In [ ]:
parms = {'verbose': 0, 'callbacks': [TQDMNotebookCallback(leave_inner=True)]}

This model here is using the previously defined blocks to encode a low resolution image and then upsample it to match the same image in high resolution.

In [ ]:
inp=Input(inp_shape)
x=conv_block(inp, 64, 9, (1,1))
for i in range(4): x=res_block(x)
x=deconv_block(x, 64, 3, (144, 144, 64))
x=deconv_block(x, 64, 3, (288, 288, 64))
x=Convolution2D(3, 9, 9, activation='tanh', border_mode='same')(x)
outp=Lambda(lambda x: (x+1)*127.5)(x)

The method of training this network is almost exactly the same as training the pixels from our previous implementations. The idea here is we're going to feed two images to Vgg16 and compare their convolutional outputs at some layer. These two images are the target image (which in our case is the same as the original but at higher resolution), and the output of the previous network we just defined, which we hope will learn to output a high resolution image. 

The key then is to train this other network to produce an image that minimizes the loss between the outputs of some convolutional layer in Vgg16 (which the paper refers to as "perceptual loss"). In doing so, we are able to train a network that can upsample an image and recreate the higher resolution details.

In [ ]:
vgg_l = Lambda(preproc)
outp_l = vgg_l(outp)

Since we only want to learn the "upsampling network", and are just using VGG to calculate the loss function, we set the Vgg layers to not be trainable.

In [ ]:
out_shape = (288,288,3)
vgg_inp=Input(out_shape)
vgg= VGG16(include_top=False, input_tensor=vgg_l(vgg_inp))
for l in vgg.layers: l.trainable=False

In [ ]:
vgg_content = Model(vgg_inp, vgg.get_layer('block2_conv2').output)
vgg1 = vgg_content(vgg_inp)
vgg2 = vgg_content(outp)

An important difference in training for super resolution is the loss function. We use what's known as a perceptual loss function (which is simply the content loss for some layer). 

In [ ]:
loss = Lambda(lambda x: K.sqrt(K.mean((x[0]-x[1])**2, (1,2))))([vgg1, vgg2])
m_final = Model([inp, vgg_inp], loss)
targ = np.zeros((arr.shape[0], 128))

Finally we compile this chain of models and we can pass it the original low resolution image as well as the high resolution to train on. We also define a zero vector as a target parameter, which is a necessary parameter when calling fit on a keras model.

In [ ]:
m_final.compile('adam', 'mse')

In [ ]:
m_final.fit([arr_lr, arr_hr], targ, 8, 2, **parms)

We use learning rate annealing to get a better fit.

In [ ]:
K.set_value(m_final.optimizer.lr, 1e-4)
m_final.fit([arr_lr, arr_hr], targ, 16, 2, **parms)

We are only interested in the trained part of the model, which does the actual upsampling.

In [ ]:
top_model = Model(inp, outp)
top_model.save_weights(dpath+'top_final.h5')

In [ ]:
p = top_model.predict(arr_lr[:20])

After training for some time, we get some very impressive results! Look at these two images, we can see that the predicted higher resolution image has filled in a lot of detail, including the shadows under the greens and the texture of the food.

In [ ]:
plt.imshow(arr_lr[10].astype('uint8'));

In [ ]:
plt.imshow(p[10].astype('uint8'))

The important thing to take away here is that as opposed to our earlier approaches, this type of approach results in a model that can created the desired image and is a scalable implementation.

Note that we haven't used a test set here, so we don't know if the above result is due to over-fitting. As part of your homework, you should create a test set, and try to train a model that gets the best result you can on the test set.

### Fast style transfer

The original paper showing the above approach to super resolution also used this approach to create a much faster style transfer system (for a specific style). Take a look at [the paper](https://arxiv.org/abs/1603.08155) and the very helpful [supplementary material](http://cs.stanford.edu/people/jcjohns/papers/eccv16/JohnsonECCV16Supplementary.pdf). Your mission, should you choose to accept it, it to modify the super resolution example above to do fast style transfer based on this paper. The supplementary material mentions that that found reflection padding helpful - we have implemented this as a keras layer for you. All the other layers and blocks are already defined above.

(This is also a nice simple example of a custom later that you can refer to when creating your own custom layers in the future.)

In [ ]:
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)
        
    def get_output_shape_for(self, s):
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

Testing the reflection padding layer:

In [ ]:
inp = Input((288,288,3))
ref_model = Model(inp, ReflectionPadding2D((40,2))(inp))
ref_model.compile('adam', 'mse')

In [ ]:
p = ref_model.predict(arr_hr[50:51])

In [ ]:
plt.imshow(p[0].astype('uint8'));

## Next steps

Here's some ideas for things to try:
- https://github.com/junyanz/iGAN
- https://arxiv.org/abs/1605.04603
- https://arxiv.org/abs/1606.05897
- http://cs231n.stanford.edu/reports2016/208_Report.pdf